In [ ]:
import os
import pandas as pd

pd.options.mode.chained_assignment = None # to suppress warnings

In [ ]:
# load

# DAF
daf = pd.read_excel("input/daf.xlsx")
# tool: survey
survey = pd.read_excel("input/tool.xlsx", sheet_name="survey")
# tool: choices
choices = pd.read_excel("input/tool.xlsx", sheet_name="choices")
# data
df = pd.read_excel("input/data.xlsx")


# specify strata 
strata = "z04_settlement"

In [ ]:
# create a list of stratas
strata_list_name = survey[survey["name"] == strata]["type"].iloc[0].split(" ")[1]
strata_list = choices[choices["list_name"] == strata_list_name]["name"].to_list()

In [ ]:
#this list will store all the generated tables
list_of_tables = []

In [ ]:
# daf_row is a currently considered row of the daf
for _, daf_row in daf.iterrows():
    # curr_var is currentrly considered vatiable in the daf
    curr_var = daf_row["variable"]
    if daf_row["func"] == "integer":
        curr_func = "integer"
    else:
        # srow is the row of the survey tab of the kobo tool with the question in the var column of the daf
        # it can be not one row if the column name contains not unique
        srow = survey[survey["name"] == daf_row["variable"]]
        # if the type is like "select_one"
        if " " in srow["type"].iloc[0]:
            curr_func, curr_list_name = srow["type"].iloc[0].split(" ")
        # nothing else is allowed
        else:
            raise ValueError("The type must be one of these: select_one, select_multiple, integer")
        # choices list for the curr_var
        choices4this = choices[choices["list_name"] == curr_list_name]["name"].to_list()
    # if we have select_one in the daf
    if curr_func == "select_one":
        # if there's no disaggregation
        if pd.isna(daf_row["disaggregations"]):
            dic_for_table = {"strata": [],
                             "total": []
                             }
            for ch in choices4this:
                dic_for_table[ch] = []
            # select only strata and curr_var and drop rows with NAs
            df_some_cols = df[[strata, curr_var]].dropna()
            total_overall = df_some_cols.shape[0]
            # if we need strata disag
            if daf_row["admin"] == "strata":
                # iterate through stratas
                for strata_i in strata_list:
                    # get only the rows within the strata_i
                    df_some_cols_strta_i = df_some_cols[df_some_cols[strata] == strata_i]
                    # total number of rows in the strata_i
                    total_strata_i = df_some_cols_strta_i.shape[0]
                    # fill the table
                    dic_for_table['strata'].append(strata_i)
                    dic_for_table['total'].append(total_strata_i)
                    # avoid division by 0
                    if total_strata_i > 0:
                        # for each choise option
                        for choice_i in choices4this:
                            # calculate the number of rows where choice_i was selected
                            num_selected_choice_i = df_some_cols_strta_i[df_some_cols_strta_i[curr_var] == choice_i].shape[0]
                            #TODO later add a switch between percentages and fractions
                            #dic_for_table[choice_i].append(num_selected_choice_i)
                            # calculate and append percentage of the choice_i
                            dic_for_table[choice_i].append("{:.2f}%".format((num_selected_choice_i / total_strata_i) * 100))
                    else:
                        for choice_i in choices4this:
                            dic_for_table[choice_i].append("0.00%")
            # overall
            dic_for_table['strata'].append("overall")
            dic_for_table['total'].append(total_overall)
            for choice_i in choices4this:
                    num_selected_choice_i = df_some_cols[df_some_cols[curr_var] == choice_i].shape[0]
                    #TODO later add a switch between percentages and fractions
                    #dic_for_table[choice_i].append(num_selected_choice_i)
                    dic_for_table[choice_i].append("{:.2f}%".format((num_selected_choice_i / total_overall) * 100))
            # turn a dictionary into dataframe
            table = pd.DataFrame(dic_for_table)
            #print(table)
            list_of_tables.append(table)
        # if there's a disaggregation
        else:
            curr_disagg = daf_row["disaggregations"]
            # select only needed columns
            df_some_cols = df[[strata, curr_var, curr_disagg]]
            # the column with disaggregations must not contain NAs
            df_some_cols[curr_disagg] = df_some_cols[curr_disagg].fillna('UNKNOWN') #TODO this line causes warnings
            # we don't care about rows where the target veriable is NA
            df_some_cols.dropna()
            disag_q_row = survey[survey["name"] == curr_disagg]
            if " " in disag_q_row["type"].iloc[0]:
                disagg_func, disag_list_name = disag_q_row["type"].iloc[0].split(" ")
            # we don't actually disaggregate by integers now. but may be we'll start later
            elif disag_q_row["type"].iloc[0] == "integer":
                disagg_func, disag_list_name = "integer", None
            else:
                raise ValueError("The type must be one of these: select_one, select_multiple, integer")
            # choices list for the disaggregation variable
            # it'll cause an error if we get integer, but we won't due to the DAF standards
            disag_list = choices[choices["list_name"] == disag_list_name]["name"].to_list()
            # add a new value of disaggregation if applicable
            if "UNKNOWN" in df_some_cols[curr_disagg].unique():
                disag_list.append("UNKNOWN")
            dic_for_table = {"strata": [],
                             curr_disagg: [],
                             "total": []
                             }
            for ch in choices4this:
                dic_for_table[ch] = []
            # if we need strata disag
            if daf_row["admin"] == "strata":
                for strata_i in strata_list:
                    # filter to get only rows for the current strata
                    df_some_cols_strta_i = df_some_cols[df_some_cols[strata] == strata_i]
                    for disag in disag_list:
                        # filter further to get rows for the current disag
                        df_some_cols_strta_i_disag = df_some_cols_strta_i[df_some_cols_strta_i[curr_disagg] == disag]
                        dic_for_table['strata'].append(strata_i)
                        dic_for_table[curr_disagg].append(disag)
                        total_disag = df_some_cols_strta_i_disag.shape[0]
                        dic_for_table['total'].append(total_disag)
                        # to avoid division by 0
                        if total_disag > 0:
                            for choice_i in choices4this:
                                num_selected_disag = df_some_cols_strta_i_disag[df_some_cols_strta_i_disag[curr_var] == choice_i].shape[0]
                                #TODO later add a switch between percentages and fractions
                                #dic_for_table[choice_i].append(num_selected_choice_i)
                                # calculate and append percentage of the choice_i
                                dic_for_table[choice_i].append("{:.2f}%".format((num_selected_disag / total_disag) * 100))
                        else:
                            for choice_i in choices4this:
                                dic_for_table[choice_i].append("0.00%")
            # overall
            for disag in disag_list:
                df_some_cols_ovarall_disag = df_some_cols[df_some_cols[curr_disagg] == disag]
                dic_for_table['strata'].append('overall')
                dic_for_table[curr_disagg].append(disag)
                total_disag = df_some_cols_ovarall_disag.shape[0]
                dic_for_table['total'].append(total_disag)
                if total_disag > 0:
                    for choice_i in choices4this:
                        num_selected_disag = df_some_cols_ovarall_disag[df_some_cols_ovarall_disag[curr_var] == choice_i].shape[0]
                        dic_for_table[choice_i].append("{:.2f}%".format((num_selected_disag / total_disag) * 100))
                else:
                    for choice_i in choices4this:
                        dic_for_table[choice_i].append("0.00%")
            table = pd.DataFrame(dic_for_table)
            #print(table)
            list_of_tables.append(table)
    elif curr_func == "integer":
        if pd.isna(daf_row["disaggregations"]):
            # for integer we calculate different statistics of the sample rather than do counts
            df_some_cols = df[[strata, curr_var]].dropna()
            total_overall = df_some_cols.shape[0]
            min_overall = df_some_cols[curr_var].min()
            max_overall = df_some_cols[curr_var].max()
            mean_overall = df_some_cols[curr_var].mean()
            median_overall = df_some_cols[curr_var].median()
            dic_for_table = {"strata": [],
                             "total": [],
                             "min": [],
                             "max": [],
                             "mean": [],
                             "median": []
                             }
            if daf_row["admin"] == "strata":
                for strata_i in strata_list:
                    df_some_cols_strta_i = df_some_cols[df_some_cols[strata] == strata_i]
                    total_strata_i = df_some_cols_strta_i.shape[0]
                    min_strata_i = df_some_cols_strta_i[curr_var].min()
                    max_strata_i = df_some_cols_strta_i[curr_var].max()
                    mean_strata_i = df_some_cols_strta_i[curr_var].mean()
                    median_strata_i = df_some_cols_strta_i[curr_var].median()
                    dic_for_table["strata"].append(strata_i)
                    dic_for_table["total"].append(total_strata_i)
                    dic_for_table["min"].append(min_strata_i)
                    dic_for_table["max"].append(max_strata_i)
                    dic_for_table["mean"].append(mean_strata_i)
                    dic_for_table["median"].append(median_strata_i)    
            dic_for_table["strata"].append("overall")
            dic_for_table["total"].append(total_overall)
            dic_for_table["min"].append(min_overall)
            dic_for_table["max"].append(max_overall)
            dic_for_table["mean"].append(mean_overall)
            dic_for_table["median"].append(median_overall)
            table = pd.DataFrame(dic_for_table)
            #print(table)
            list_of_tables.append(table)
        else:
            #TODO implement disaggregation
            curr_disagg = daf_row["disaggregations"]
            df_some_cols = df[[strata, curr_var, curr_disagg]]
            df_some_cols[curr_disagg] = df_some_cols[curr_disagg].fillna('UNKNOWN')
            df_some_cols.dropna()
            disag_q_row = survey[survey["name"] == curr_disagg]
            if " " in disag_q_row["type"].iloc[0]:
                disagg_func, disag_list_name = disag_q_row["type"].iloc[0].split(" ")
            # we don't disaggregate by integer yet
            elif disag_q_row["type"].iloc[0] == "integer":
                disagg_func, disag_list_name = "integer", None
            else:
                raise ValueError("The type must be one of these: select_one, select_multiple, integer")
            # choices list for the disaggregation variable
            disag_list = choices[choices["list_name"] == disag_list_name]["name"].to_list()
            if "UNKNOWN" in df_some_cols[curr_disagg].unique():
                disag_list.append("UNKNOWN")
            dic_for_table = {"strata": [],
                             curr_disagg: [],
                             "total": [],
                             "min": [],
                             "max": [],
                             "mean": [],
                             "median": []
                             }
            if daf_row["admin"] == "strata":
                for strata_i in strata_list:
                    df_some_cols_strta_i = df_some_cols[df_some_cols[strata] == strata_i]
                    for disag in disag_list:
                        df_some_cols_strta_i_disag = df_some_cols_strta_i[df_some_cols_strta_i[curr_disagg] == disag]
                        total_strta_i_disag = df_some_cols_strta_i_disag.shape[0]
                        min_strta_i_disag = df_some_cols_strta_i_disag[curr_var].min()
                        max_strta_i_disag = df_some_cols_strta_i_disag[curr_var].max()
                        mean_strta_i_disag = df_some_cols_strta_i_disag[curr_var].mean()
                        median_strta_i_disag = df_some_cols_strta_i_disag[curr_var].median()
                        dic_for_table["strata"].append(strata_i)
                        dic_for_table[curr_disagg].append(disag)
                        dic_for_table["total"].append(total_strta_i_disag)
                        dic_for_table["min"].append(min_strta_i_disag)
                        dic_for_table["max"].append(max_strta_i_disag)
                        dic_for_table["mean"].append(mean_strta_i_disag)
                        dic_for_table["median"].append(median_strta_i_disag)
            for disag in disag_list:
                df_some_cols_disag = df_some_cols[df_some_cols[curr_disagg] == disag]
                total_disag = df_some_cols_disag.shape[0]
                min_disag = df_some_cols_disag[curr_var].min()
                max_disag = df_some_cols_disag[curr_var].max()
                mean_disag = df_some_cols_disag[curr_var].mean()
                median_disag = df_some_cols_disag[curr_var].median()
                dic_for_table["strata"].append("overall")
                dic_for_table[curr_disagg].append(disag)
                dic_for_table["total"].append(total_disag)
                dic_for_table["min"].append(min_disag)
                dic_for_table["max"].append(max_disag)
                dic_for_table["mean"].append(mean_disag)
                dic_for_table["median"].append(median_disag)
            table = pd.DataFrame(dic_for_table)
            #print(table)
            list_of_tables.append(table)

In [ ]:
with pd.ExcelWriter("output/tables.xlsx", engine='xlsxwriter') as writer:
    # Loop through the list of DataFrames and save them in different sheets
    for idx in range(len(list_of_tables)):
        list_of_tables[idx].to_excel(writer, sheet_name='table_{}'.format(idx+1), index=False)
    